# Example: Solving Ordinary Differential Equations

In this notebook we will use Python to solve differential equations numerically.

In [ ]:
// Import the required modules
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FSharp.Stats, 0.4.8"
#r "nuget: FsODE, 0.0.2"

// ... and open the required modules
open FSharp.Stats
open FsODE
open FsODE.Stochastic
open Plotly.NET

Installed Packages FSharp.Stats, 0.4.8 FsODE, 0.0.2 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

## Stichproben aus Verteilungen

Für einige Situationen kennen wir die Wahrscheinlichkeitsverteilung. Wenn die Situation zum Beispiel so aussieht, dass wir n
unabhängige Münzwürfe machen, jeder mit der Wahrscheinlichkeit p, dass Kopf fällt, dann ist die Anzahl h
der Köpfe binomialverteilt. Wir könnten beweisen, dass dies der Fall ist, oder es in einem Buch nachschlagen. Aber manchmal ist es zu schwierig (oder unmöglich) zu beweisen, wie die Verteilung aussieht. Daher können wir ihre Eigenschaften numerisch berechnen, indem wir eine Stichprobe aus der Verteilung nehmen. Für die Stichprobe wird ein Zufallszahlengenerator verwendet, um die Situation zu simulieren, aus der die Verteilung resultiert. Wenn Sie also die Situation kennen und einen Computer zur Hand haben, können Sie über eine Stichprobe ein Diagramm der Verteilung erhalten.

Lassen Sie uns dies anhand der Binomialverteilung demonstrieren. Wir nehmen n=25
und p=0.25 und berechnen P(h∣n,p), die Wahrscheinlichkeit, dass h Köpfe bei n Münzwürfen mit einer Wahrscheinlichkeit p für Köpfe auftreten. Wir ziehen 10, 30, 100 und 300 Stichproben und stellen sie der erwarteten Binomialverteilung gegenüber.

In [ ]:
let stichproben = [|30;100;1000;10000|]
let n = 25
let p = 0.25

let binomialverteilung = Distributions.Discrete.Binomial.Init 0.25 25

let binomialPunkte =
    [|0 .. 25|]
    |> Array.map (fun x -> x,binomialverteilung.PMF x)

stichproben
|> Array.map (fun s ->
    [
        simulateCoinflips n p s
        |> Array.countBy id
        |> Array.map (fun (heads,count) -> heads, float count/ float s)
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromString "Orange")
        binomialPunkte
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromString "Blue", Opacity = 0.3)
    ]
    |> Chart.combine
    |> Chart.withLegend false
    |> Chart.withXAxisStyle(TitleText = $"h<br>{s} Stichproben")
    |> Chart.withYAxisStyle(TitleText = "P(h)")
)
|> Chart.Grid(2,2)
|> Chart.withSize (800,800)

<!-- Plotly chart will be drawn inside this DIV -->

### Aufgabe

Die meisten Münzen haben eine Wahrscheinlichkeit von p=0.5 für Kopf. Simulieren Sie diese Situation wie wie oben gezeigt und vergleichen Sie die Verteilungen aus den unterschiedlichen Stichprobenzahlen mit der tatsächlichen Binomialverteilung.

In [ ]:
// Kopieren sie den benötigten Code aus dem vorherigen Codeblock
let stichproben = [|30;100;1000;10000|]
let n = 25
let p = 0.5

let binomialverteilung = Distributions.Discrete.Binomial.Init 0.5 25
let binomialPunkte =
    [|0 .. 25|]
    |> Array.map (fun x -> x,binomialverteilung.PMF x)

stichproben
|> Array.map (fun s ->
    [
        simulateCoinflips n p s
        |> Array.countBy id
        |> Array.map (fun (heads,count) -> heads, float count/ float s)
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromString "Orange")
        binomialPunkte
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromString "Blue", Opacity = 0.3)
    ]
    |> Chart.combine
    |> Chart.withLegend false
    |> Chart.withXAxisStyle(TitleText = $"h<br>{s} Stichproben")
    |> Chart.withYAxisStyle(TitleText = "P(h)")
)
|> Chart.Grid(2,2)
|> Chart.withSize (800,800)

<!-- Plotly chart will be drawn inside this DIV -->

Wie wir sehen, können wir die tatsächliche Verteilung annähernd berechnen, wenn wir eine Stichprobe aus der Wahrscheinlichkeitsverteilung ziehen. Je mehr Stichproben wir nehmen, desto besser wird die Annäherung.
Stichproben sind eine so leistungsfähige Strategie, dass hocheffiziente Algorithmen mit praktischen APIs entwickelt wurden, um Stichproben aus benannten Wahrscheinlichkeitsverteilungen zu ziehen. Wir hätten zum Beispiel `binomialverteilung.Sample()` als einfachen (und viel effizienteren) Ersatz für die obige Funktion `simulateCoinflips` verwenden können.

Wir werden die gleiche Strategie für die Lösung von stochastischen Gleichungssystemen verwenden. Wir werden eine stochastische Methode verwenden, um eine Stichprobe aus der Verteilung zu ziehen, die von dem Gleichungssystem beschrieben wird. Diese Methode wurde in den letzten 70er Jahren von Dan Gillespie entwickelt. Aus diesem Grund werden diese Stichprobenverfahren oft als Gillespie-Simulationen bezeichnet. Der Algorithmus wird manchmal auch als stochastischer Simulationsalgorithmus oder SSA bezeichnet.

Im Folgenden werden wir die Funktionsweise dieses Algorithmus anhand der einfachen Produktion eines Proteins untersuchen.

## Simulieren des stochastischen Gleichungssystems

Für die einfache Proteinproduktion gibt es die folgende Reaktion:

DNA→mRNA→Protein

Wir haben bereits vorher die Proteinproduktion mit Hilfe von Differentialgleichungen simuliert. Das Ergebnis davon ist jedoch nicht für jeden Fall anwendbar. Was wir bei dieser Simulation gesehen haben, ist der Mittelwert der Produktion bei einer großen Zahl an Reaktionen. Sobald aber stochastische Effekte eine größere Rolle spielen (z.B. das Gesetz der großen Zahlen nicht anwendbar ist und mit wenig Reaktionen gearbeitet wird), sind stochastische Simulationen eine geeignetere Methode.

## Anwenden der Gillespie Simulation

Um die Gillespie-Simulation zu verwenden, erstellen wir zunächst ein Array, das die Änderungen in den Zahlen von mRNA (m)
und Protein (p) für jede der vier möglichen Reaktionen enthält. Auf diese Art und Weise werden die Aktualisierungen der Teilchenzahlen kodiert, die wir durch die Wahl der jeweiligen Zustandsänderungen erhalten.

In [ ]:
// Index 0 repräsentiert Änderung bei mRNA (m), Index 1 repräsentiert Änderung bei Protein (p)
let simpleUpdate =
    [|
        1., 0.;  // Make mRNA transcript
        -1., 0.; // Degrade mRNA
        0., 1.;  // Make protein
        0., -1.  // Degrade protein
    |]

Zusätzlich benötigen wir die Übergangswahrscheinlichkeiten der Zustandsänderungen. Die Übergangswahrscheinlichkeiten werden im Rahmen der stochastischen Simulation auch als Propensities bezeichnet.
Dafür erstellen wir eine Funktion, die das Array der Propensities für jede der vier Reaktionen aktualisiert. Wir aktualisieren die Propensities (die als Argument an die Funktion übergeben werden), anstatt sie zu instanziieren und zurückzugeben, um bei der Ausführung des Codes Speicherplatz zu sparen. Sie wird eine Funktion der aktuellen Anzahl von Molekülen sein. Im Allgemeinen kann sie auch eine Funktion der Zeit sein, daher berücksichtigen wir ausdrücklich auch die Zeitabhängigkeit (auch wenn wir sie in diesem einfachen Beispiel nicht verwenden).

In [ ]:
//Updates einen Array von Propensities abhängig von den Parametern und der Population
let simplePropensities (propensities: float []) population t beta_m beta_p gamma =
    let m,p = population
    propensities.[0] <- beta_m     // Make mRNA transcript
    propensities.[1] <- m          // Degrade mRNA
    propensities.[2] <- beta_p * m // Make protein
    propensities.[3] <- gamma * p  // Degrade protein

## "Ziehen" einer Reaktion

Anschließend brauchen wir eine allgemeine Funktion, die eine bestimmte Reaktion und das Zeitintervall für diese Reaktion zieht. Um das Zeitintervall zu bestimmen, ziehen wir zunächst eine Zufallszahl aus einer Exponentialverteilung.
Als nächstes müssen wir auswählen, welche Reaktion stattfinden soll. Das läuft darauf hinaus, eine Stichprobe aus einer diskreten Verteilung zu ziehen.
Dabei ist die Wahrscheinlichkeit jeder Reaktion proportional zu ihrer Propensity.

Hier können sie die dafür verwendete Funktion ausprobieren. Die Funktion gibt mit der an dem jeweiligen Index angegebenen Wahrscheinlichkeit den Index bei Aufruf der Funktion zurück.

In [ ]:
// Make dummy probs
let probs = [|0.1;0.3;0.4;0.05;0.15|]
sampleDiscrete probs

2

### Aufgabe

Führen sie die Funktion `sampleDiscrete` mit Wahrscheinlichkeiten Ihrer Wahl mehrfach aus. Stimmen die Ziehungen der Indices mit den Angegeben Wahrscheinlichkeiten überein?

## SSA Zeitsprungverfahren

Jetzt können wir unsere SSA-Hauptschleife verwenden. Wir werden nur die Zahlen zu vorher festgelegten Zeitpunkten speichern. Das spart Arbeitsspeicher, und wir interessieren uns ohnehin nur für die Werte zu bestimmten Zeitpunkten.

Beachten Sie, dass diese Funktion generisch ist. Alles, was wir brauchen, um unser System zu spezifizieren, ist das Folgende:

    - Eine Funktion zum Berechnen der Übergangswahrscheinlichkeiten
    - Wie die Aktualisierungen für eine bestimmte Reaktion vorgenommen werden
    - Ausgangspopulation

Zusätzlich geben wir die notwendigen Parameter, eine Anfangsbedingung und die Zeitpunkte an, zu denen wir unsere Stichproben speichern wollen.

Wir können nun eine Reihe von SSA-Simulationen durchführen und die Ergebnisse speichern. Wir führen dafür 100 Trajektorien mit den Werten βp=βm=10 und γ=0,4 durch.

In [ ]:
let zeitpunkte = [|0. .. 0.5 .. 50.|]

let samples =
    [|
        for i = 0 to 1000 do
            gillespieSSA simplePropensities simpleUpdate (0,0) zeitpunkte 10. 10. 0.4
    |]

In [ ]:
[
    samples
    |> Array.map fst
    |> Array.transpose
    |> Array.map (fun numbers -> Array.average numbers, Seq.stDev numbers)
    |> Array.unzip
    |> fun (number,stDev) ->
        Chart.Line (zeitpunkte, number, LineWidth = 5)
        |> Chart.withYErrorStyle(Array = stDev, Color = Color.fromString "grey")
    samples
    |> Array.map snd
    |> Array.transpose
    |> Array.map (fun numbers -> Array.average numbers, Seq.stDev numbers)
    |> Array.unzip
    |> fun (number,stDev) ->
        Chart.Line (zeitpunkte, number, LineWidth = 5)
        |> Chart.withYErrorStyle(Array = stDev, Color = Color.fromString "grey")
]
|> Chart.Grid (1,2)
|> Chart.withSize(1200., 600.)

<!-- Plotly chart will be drawn inside this DIV -->

### Aufgabe
Führen sie `gillespieSSA` mit den obigen Werten unterschiedlich oft aus. Wie sehen die Kurven bei niedriger Wiederholungszahl aus? Wie bei hoher?

In [ ]:
// Kopieren sie den benötigten Code aus dem vorherigen Codeblock

Abschließend können wir die Wahrscheinlichkeitsverteilungen im stationären Zustand anschauen. Dafür schauen wir uns die Häufigkeiten der einzelnen Werte als normiertes Histogramm mit Bingröße=1 an. Dies entspricht der Wahrscheinlichkeitsdichtefunktion (PDF).

In [ ]:
let samplesmRNA =
    samples
    |> Array.map fst
    |> Array.collect (fun x -> x.[50 ..])

let sampleZahlenmRNA =
    samplesmRNA
    |> Array.countBy id
    |> fun histo ->
        let histoSum =
            histo
            |> Array.map snd
            |> Array.sum
        histo
        |> Array.map (fun (event,count) -> event, float count/ float histoSum)

let histomRNA =
    Chart.Column(sampleZahlenmRNA)
    |> Chart.withXAxisStyle(TitleText = "mRNA Anzahl")
    |> Chart.withYAxisStyle(TitleText = "P(mRNA)")

histomRNA

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let samplesProtein =
    samples
    |> Array.map snd
    |> Array.collect (fun x -> x.[50 ..])

let sampleZahlenProtein =
    samplesProtein
    |> Array.countBy id
    |> fun histo ->
        let histoSum =
            histo
            |> Array.map snd
            |> Array.sum
        histo
        |> Array.map (fun (event,count) -> event, float count/ float histoSum)

let histoProtein =
    Chart.Column(sampleZahlenProtein)
    |> Chart.withXAxisStyle(TitleText = "Protein Anzahl")
    |> Chart.withYAxisStyle(TitleText = "P(Protein)")

histoProtein

<!-- Plotly chart will be drawn inside this DIV -->

### Aufgabe
Welcher Verteilung entspricht die mRNA-, und welcher die Protein-Verteilung? Probieren sie die Verteilungen aus und bestimmen sie die passenden Parameter.

In [ ]:
let poissonLambda = Distributions.Discrete.Poisson.Fit samplesmRNA
let poissonVerteilung = Distributions.Discrete.Poisson.Init poissonLambda

[
    histomRNA
    Chart.Line ([|0. .. 25.|], [|0 .. 25|] |> Array.map poissonVerteilung.PMF)
]
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let gammaAlpha,gammaBeta = Distributions.Continuous.Gamma.Fit samplesProtein
let gammaVerteilung = Distributions.Continuous.Gamma.Init gammaAlpha gammaBeta

[
    histoProtein
    Chart.Line ([|0. .. 500.|], [|0. .. 500.|] |> Array.map gammaVerteilung.PDF)
]
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->